<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
IC TABLES - GENERATION</p><br>

In [90]:
# General importings
import sys,os,os.path

sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/Users/halmamol/NEXT/IC'
sys.path.insert(1, '/Users/halmamol/NEXT/IC')

import json
import random

import invisible_cities.database.load_db as db

import sqlite3
from glob import glob

In [60]:
# Detectors: "Tst", "Next100", "Flex100", "FlexDens"

det_name = "FlexDens10"

In [61]:
tables = [
    "ChannelGain",
    "ChannelMapping",
    "ChannelMask",
    "ChannelPosition",
    "DetectorGeo",
    "PMTFELowFrequencyNoise",
    "PMTFEMapping",
    "PmtBlr",
    "PmtNoiseRms",
    "SipmBaseline",
    "SipmNoisePDF"
]

In [62]:
def load_detector_config(det_name):
    config_fname = f"config/{det_name}.config"

    with open(config_fname) as config_file:
        det_config = json.load(config_file)
    
    # Getting sensors info
    sensor_labels = det_config["SensorLabels"]
    for label in sensor_labels:
        det_config[label] = []
    
    sensors_fname = det_config["SensorFile"]
    with open(f"config/{sensors_fname}", 'r') as sensors_file:
        for entry in sensors_file.read().splitlines():
            words = entry.split()
            if (len(words) and (words[0] in sensor_labels)):
                sensor_id = int(words[1])
                pos = words[2][1:-1].split(',')
                sensor_x = float(pos[0])
                sensor_y = float(pos[1])
                det_config[words[0]].append((sensor_id, sensor_x, sensor_y))

    return det_config

In [63]:
def get_ChannelGain_entries(det_config):
    
    template = open("templates/ChannelGain.entry").read()
    
    
    MinRun = det_config["MinRun"]
    MaxRun = det_config["MaxRun"]
    
    entries = ""
    for label in det_config["SensorLabels"]:
        entries += "\n"
        for sensor in det_config[label]:
            SensorID      = sensor[0]
            Centroid      = det_config[label + "_Centroid"]
            ErrorCentroid = det_config[label + "_ErrorCentroid"]
            Sigma         = det_config[label + "_Sigma"]
            ErrorSigma    = det_config[label + "_ErrorSigma"]
            params        = locals()
            entries      += template.format(**params) + "\n"

    return entries

In [64]:
def get_ChannelMapping_entries(det_config):
    
    template = open("templates/ChannelMapping.entry").read()
    
    MinRun = det_config["MinRun"]
    MaxRun = det_config["MaxRun"]

    entries = ""
    for label in det_config["SensorLabels"]:
        entries += "\n"
        for sensor in det_config[label]:
            SensorID = sensor[0]
            ElecID   = SensorID
            params   = locals()
            entries += template.format(**params) + "\n"

    return entries

In [65]:
def get_ChannelPosition_entries(det_config):
    
    template = open("templates/ChannelPosition.entry").read()
    
    MinRun = det_config["MinRun"]
    MaxRun = det_config["MaxRun"]

    entries = ""
    for label in det_config["SensorLabels"]:
        entries += "\n"
        for sensor in det_config[label]:
            SensorID      = sensor[0]
            Label         = label
            Type          = det_config[label + "_Type"]
            X             = sensor[1]
            Y             = sensor[2]
            params        = locals()
            entries      += template.format(**params) + "\n"

    return entries

In [66]:
def get_DetectorGeo_entries(det_config):
    
    template = open("templates/DetectorGeo.entry").read()
    
    XMIN = det_config["XMIN"]
    XMAX = det_config["XMAX"]
    YMIN = det_config["YMIN"]
    YMAX = det_config["YMAX"]
    ZMIN = det_config["ZMIN"]
    ZMAX = det_config["ZMAX"]
    RMAX = det_config["RMAX"]

    params  = locals()
    entries = template.format(**params) + "\n"

    return entries

In [67]:
def get_SipmBaseline_entries(det_config):
    
    template = open("templates/SipmBaseline.entry").read()
    
    MinRun = det_config["MinRun"]
    MaxRun = "NULL"

    entries = ""
    for label in det_config["SensorLabels"]:
        if "SiPM" in label:
            entries += "\n"
            for sensor in det_config[label]:
                SensorID = sensor[0]
                Energy   = det_config[label + "_BaseLine"]
                params   = locals()
                entries += template.format(**params) + "\n"

    return entries

In [68]:
def generate_SipmNoisePDF_DB(tPath, table_fname):
    
    table = "SipmNoisePDF"
    print(f"Generating table {table} ...")

    #det_config = load_detector_config(det_name)
    
    # Getting header
    header_fname = f"{tPath}/{table}.h"
    with open(header_fname, 'r') as header_file:
        header = header_file.read() + "\n"
    with open(table_fname, 'w') as table_file:
        table_file.write(header)

    # Generating entries
    template = open("templates/SipmNoisePDF.entry").read()
    MinRun = det_config["MinRun"]
    MaxRun = "NULL"
    num_sipms = 0
    entries   = ""
    for label in det_config["SensorLabels"]:
        if "SiPM" in label:            
            noiseDB = det_config[label + "_NoiseDB"]
            noise, noise_bins, baselines = db.SiPMNoise(noiseDB[0], noiseDB[1])
            
            sipm_data    = db.DataSiPM(noiseDB[0], noiseDB[1])
            masked_sipms = sipm_data[sipm_data.Active == 0].index.to_numpy()
            
            for sensor in det_config[label]:
                num_sipms += 1
                SensorID   = sensor[0]
                # Randomly select a NEW SiPM pdf
                NEW_sensor_id = random.randint(0, 1791)
                while NEW_sensor_id in masked_sipms:
                    NEW_sensor_id = random.randint(0, 1791)
                noise_pdf = noise[NEW_sensor_id]
                
                for BinEnergyPes, Probability in zip(noise_bins, noise_pdf):
                    params   = locals()
                    entries += template.format(**params) + "\n"
                
                if(num_sipms % 10 == 0):
                    print(f"  SiPMs managed: {num_sipms}")
                    with open(table_fname, 'a') as table_file:
                        table_file.write(entries)
                    entries=''

    ### Writing table
    with open(table_fname, 'a') as table_file:
        table_file.write(entries)

    print(f"Total SiPMs managed: {num_sipms}")

In [75]:
### Reading Detector Configuration
det_config = load_detector_config(det_name)


### PATHs
tPath = "./templates"
oPath = f"{det_name}"
if not os.path.isdir(oPath):
    os.makedirs(oPath)


### Generating tables
for table in tables:
    
    print(f"Generating table {table} ...")
    table_fname  = f"{oPath}/{table}.sql"

    if table == "SipmNoisePDF":
        generate_SipmNoisePDF_DB(tPath, table_fname)
    else:
        # Getting header
        header_fname = f"{tPath}/{table}.h"
        with open(header_fname, 'r') as header_file:
            content = header_file.read() + "\n"

        # Add table registers
        if table   == "ChannelGain"    : content += get_ChannelGain_entries(det_config)
        elif table == "ChannelMapping" : content += get_ChannelMapping_entries(det_config)
        elif table == "ChannelPosition": content += get_ChannelPosition_entries(det_config)
        elif table == "DetectorGeo"    : content += get_DetectorGeo_entries(det_config)
        elif table == "SipmBaseline"   : content += get_SipmBaseline_entries(det_config)

        # Writing table
        with open(table_fname, 'w') as table_file:
            table_file.write(content)

print("Done !!")

Generating table ChannelGain ...
Generating table ChannelMapping ...
Generating table ChannelMask ...
Generating table ChannelPosition ...
Generating table DetectorGeo ...
Generating table PMTFELowFrequencyNoise ...
Generating table PMTFEMapping ...
Generating table PmtBlr ...
Generating table PmtNoiseRms ...
Generating table SipmBaseline ...
Generating table SipmNoisePDF ...
Generating table SipmNoisePDF ...
  SiPMs managed: 10
  SiPMs managed: 20
  SiPMs managed: 30
  SiPMs managed: 40
  SiPMs managed: 50
  SiPMs managed: 60
  SiPMs managed: 70
  SiPMs managed: 80
  SiPMs managed: 90
  SiPMs managed: 100
  SiPMs managed: 110
  SiPMs managed: 120
  SiPMs managed: 130
  SiPMs managed: 140
  SiPMs managed: 150
  SiPMs managed: 160
  SiPMs managed: 170
  SiPMs managed: 180
  SiPMs managed: 190
  SiPMs managed: 200
  SiPMs managed: 210
  SiPMs managed: 220
  SiPMs managed: 230
  SiPMs managed: 240
  SiPMs managed: 250
  SiPMs managed: 260
  SiPMs managed: 270
  SiPMs managed: 280
  SiPMs 

  SiPMs managed: 3600
  SiPMs managed: 3610
  SiPMs managed: 3620
  SiPMs managed: 3630
  SiPMs managed: 3640
  SiPMs managed: 3650
  SiPMs managed: 3660
  SiPMs managed: 3670
  SiPMs managed: 3680
  SiPMs managed: 3690
  SiPMs managed: 3700
  SiPMs managed: 3710
  SiPMs managed: 3720
  SiPMs managed: 3730
  SiPMs managed: 3740
  SiPMs managed: 3750
  SiPMs managed: 3760
  SiPMs managed: 3770
  SiPMs managed: 3780
  SiPMs managed: 3790
  SiPMs managed: 3800
  SiPMs managed: 3810
  SiPMs managed: 3820
  SiPMs managed: 3830
  SiPMs managed: 3840
  SiPMs managed: 3850
  SiPMs managed: 3860
  SiPMs managed: 3870
  SiPMs managed: 3880
  SiPMs managed: 3890
  SiPMs managed: 3900
  SiPMs managed: 3910
  SiPMs managed: 3920
  SiPMs managed: 3930
  SiPMs managed: 3940
  SiPMs managed: 3950
  SiPMs managed: 3960
  SiPMs managed: 3970
  SiPMs managed: 3980
  SiPMs managed: 3990
  SiPMs managed: 4000
  SiPMs managed: 4010
  SiPMs managed: 4020
  SiPMs managed: 4030
  SiPMs managed: 4040
  SiPMs ma

  SiPMs managed: 7340
  SiPMs managed: 7350
  SiPMs managed: 7360
  SiPMs managed: 7370
  SiPMs managed: 7380
  SiPMs managed: 7390
  SiPMs managed: 7400
  SiPMs managed: 7410
  SiPMs managed: 7420
  SiPMs managed: 7430
  SiPMs managed: 7440
  SiPMs managed: 7450
  SiPMs managed: 7460
  SiPMs managed: 7470
  SiPMs managed: 7480
Total SiPMs managed: 7484
Done !!


In [76]:
dbfile = f'localdb.{det_name}.sqlite3'
print(f'Output file sqlite3 {dbfile}')
print("Getting sql files")
conn_sqlite   = sqlite3.connect(dbfile)
cursor_sqlite = conn_sqlite.cursor()
sql_files = glob(f'{det_name}/*sql')
sql_files

Output file sqlite3 localdb.FlexDens10.sqlite3
Getting sql files


['FlexDens10/ChannelMapping.sql',
 'FlexDens10/PMTFELowFrequencyNoise.sql',
 'FlexDens10/ChannelPosition.sql',
 'FlexDens10/PmtBlr.sql',
 'FlexDens10/ChannelGain.sql',
 'FlexDens10/SipmBaseline.sql',
 'FlexDens10/DetectorGeo.sql',
 'FlexDens10/PMTFEMapping.sql',
 'FlexDens10/SipmNoisePDF.sql',
 'FlexDens10/ChannelMask.sql',
 'FlexDens10/PmtNoiseRms.sql']

In [80]:
for sqlfile in sql_files:
    print(sqlfile)
    with open(sqlfile) as fd:
        data = fd.read()
        sentences = data.split(';')
        for sentence in sentences:
            cursor_sqlite.execute(sentence)
        print('-> done!')
        conn_sqlite.commit()

FlexDens10/ChannelMapping.sql
-> done!
FlexDens10/PMTFELowFrequencyNoise.sql
-> done!
FlexDens10/ChannelPosition.sql
-> done!
FlexDens10/PmtBlr.sql
-> done!
FlexDens10/ChannelGain.sql
-> done!
FlexDens10/SipmBaseline.sql
-> done!
FlexDens10/DetectorGeo.sql
-> done!
FlexDens10/PMTFEMapping.sql
-> done!
FlexDens10/SipmNoisePDF.sql
-> done!
FlexDens10/ChannelMask.sql
-> done!
FlexDens10/PmtNoiseRms.sql
-> done!


In [93]:
db.DataSiPM('flexdens10').head()

,SensorID,ChannelID,Active,X,Y,adc_to_pes,Sigma
0,1000,1000,1,-45.0,-485.0,17.25,1.677
1,1001,1001,1,-35.0,-485.0,17.25,1.677
2,1002,1002,1,-25.0,-485.0,17.25,1.677
3,1003,1003,1,-15.0,-485.0,17.25,1.677
4,1004,1004,1,-5.0,-485.0,17.25,1.677


In [95]:
db.DataSiPM('flexdens5').head()

,SensorID,ChannelID,Active,X,Y,adc_to_pes,Sigma
0,1000,1000,1,-17.5,-487.5,17.25,1.677
1,1001,1001,1,-12.5,-487.5,17.25,1.677
2,1002,1002,1,-7.5,-487.5,17.25,1.677
3,1003,1003,1,-2.5,-487.5,17.25,1.677
4,1004,1004,1,2.5,-487.5,17.25,1.677
